In [1]:
# Import and create a new SQLContext 
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
# Read the country CSV file into an RDD.
country_lines = sc.textFile('file:///home/cloudera/Downloads/big-data-3/final-project/country-list.csv')

In [53]:
# Convert each line into a pair of words


In [62]:
# Convert each pair of words into a tuple
country_tuples = country_lines.map(lambda line: (line.split(", ")[0], line.split(", ")[1]))

In [61]:
for x in a.collect():
    print(x)

('Afghanistan', 'AFG')
('Albania', 'ALB')
('Algeria', 'ALG')
('American Samoa', 'ASA')
('Andorra', 'AND')
('Angola', 'ANG')
('Anguilla', 'AIA')
('Antigua and Barbuda', 'ATG')
('Argentina', 'ARG')
('Armenia', 'ARM')
('Aruba', 'ARU')
('Australia', 'AUS')
('Austria', 'AUT')
('Azerbaijan', 'AZE')
('Bahamas', 'BAH')
('Bahrain', 'BHR')
('Bangladesh', 'BAN')
('Barbados', 'BRB')
('Belarus', 'BLR')
('Belgium', 'BEL')
('Belize', 'BLZ')
('Benin', 'BEN')
('Bermuda', 'BER')
('Bhutan', 'BHU')
('Bolivia', 'BOL')
('Bosnia and Herzegovina', 'BIH')
('Botswana', 'BOT')
('Brazil', 'BRA')
('British Virgin Islands', 'VGB')
('Brunei', 'BRU')
('Bulgaria', 'BUL')
('Burkina Faso', 'BFA')
('Burundi', 'BDI')
('Cambodia', 'CAM')
('Cameroon', 'CMR')
('Canada', 'CAN')
('Cape Verde', 'CPV')
('Cayman Islands', 'CAY')
('Central African Republic', 'CTA')
('Chad', 'CHA')
('Chile', 'CHI')
('China PR', 'CHN')
('Chinese Taipei', 'TPE')
('Colombia', 'COL')
('Comoros', 'COM')
('Congo', 'CGO')
('DR Congo', 'COD')
('Cook Island

In [63]:
# Create the DataFrame, look at schema and contents
countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])
countryDF.printSchema()
countryDF.take(3)

root
 |-- country: string (nullable = true)
 |-- code: string (nullable = true)



[Row(country='Afghanistan', code='AFG'),
 Row(country='Albania', code='ALB'),
 Row(country='Algeria', code='ALG')]

In [79]:
# Read tweets CSV file into RDD of lines
tweets_lines = sc.textFile('file:///home/cloudera/Downloads/big-data-3/final-project/tweet_out2.csv')

In [82]:
tweets_lines.flatMap(lambda x: x.split(",")).collect()

['tweet_text',
 'user.Location',
 'RT @ochocinco: I beat them all for 10 straight hours #FIFA16KING  https://t.co/BFnV6jfkBL',
 '501',
 'RT @NiallOfficial: @Louis_Tomlinson @socceraid when I retired from playing because of my knee . I went and did my uefa A badges in Dublin',
 '',
 'RT @GameSeek: Follow & Retweet for your chance to win a copy of FIFA 17 Deluxe Edition (platform of your choice) in our #giveaway! https://…',
 '',
 '@CIVARAGI ...I was putting ffs but it autocorrected it too FIFA',
 '',
 "RT @GeniusFootball: You know it's FIFA... https://t.co/tLK6sTnPaM",
 '',
 '"RT @WeahsCousin: ""Pogba isn\'t worth £100million.""',
 '',
 'Thanks Dean',
 ' the £500 you\'ve spent on FIFA Ultimate Team probably wasn\'t worth it either."',
 'Last Laugh Lane',
 '"RT @WeahsCousin: ""Pogba isn\'t worth £100million.""',
 '',
 'Thanks Dean',
 ' the £500 you\'ve spent on FIFA Ultimate Team probably wasn\'t worth it either."',
 'USA',
 'New on eBay! XBOX 360 Game FIFA 16 2016 https://t.co/xicyLOE6a

In [83]:
# Clean the data: some tweets are empty. Remove the empty tweets using filter() 
tweets_lines_clean = tweets_lines.filter(lambda x: len(x)>0)

In [84]:
print(tweets_lines.count())
print(tweets_lines_clean.count())

14000
13396


In [75]:
# Perform WordCount on the cleaned tweet texts. (note: this is several lines.)
tweets_wordcount = tweets_lines_clean.flatMap(lambda x: x.split(" ")) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda a, b: (a + b))

In [112]:
# Create the DataFrame of tweet word counts
tweetsDF = sqlContext.createDataFrame(tweets_wordcount, ['word', 'count'])

In [113]:
countryDF.show(5)
tweetsDF.show(5)

+--------------+----+
|       country|code|
+--------------+----+
|   Afghanistan| AFG|
|       Albania| ALB|
|       Algeria| ALG|
|American Samoa| ASA|
|       Andorra| AND|
+--------------+----+
only showing top 5 rows

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    | 3292|
|https://t.co/fQft...|    1|
|              mobile|    1|
|#FridayNightTouch...|    1|
|              circle|    7|
+--------------------+-----+
only showing top 5 rows



In [111]:
countryList = [i.country for i in countryDF.select('country').collect()]


Column<b'_1'>

In [114]:
# Join the country and tweet data frames (on the appropriate column)
merge = countryDF.join(tweetsDF, countryDF.country == tweetsDF.word)

In [120]:
# Question 1: number of distinct countries mentioned
merge.filter(merge['count']>0).count()

44

In [128]:
# Question 2: number of countries mentioned in tweets.
from pyspark.sql.functions import sum
merge.groupBy().sum("count").collect()

[Row(sum(count)=397)]

In [129]:
# Table 1: top three countries and their counts.
from pyspark.sql.functions import desc

merge.sort('count', ascending=False).show()


+-----------+----+-----------+-----+
|    country|code|       word|count|
+-----------+----+-----------+-----+
|     Norway| NOR|     Norway|   52|
|    Nigeria| NGA|    Nigeria|   49|
|     France| FRA|     France|   42|
|   Slovakia| SVK|   Slovakia|   30|
|    England| ENG|    England|   25|
|    Germany| GER|    Germany|   20|
|      Wales| WAL|      Wales|   19|
|     Russia| RUS|     Russia|   15|
|     Brazil| BRA|     Brazil|   13|
|Netherlands| NED|Netherlands|   13|
|     Canada| CAN|     Canada|   11|
|Switzerland| SUI|Switzerland|   10|
|       Chad| CHA|       Chad|    9|
|     Guinea| GUI|     Guinea|    8|
|      Spain| ESP|      Spain|    8|
|   Portugal| POR|   Portugal|    8|
|     Jordan| JOR|     Jordan|    6|
|       Iraq| IRQ|       Iraq|    6|
|      Japan| JPN|      Japan|    5|
|    Austria| AUT|    Austria|    5|
+-----------+----+-----------+-----+
only showing top 20 rows



In [ ]:
# Table 2: counts for Wales, Iceland, and Japan.


In [130]:
merge.groupBy().avg("count").collect()

[Row(avg(count)=9.022727272727273)]